# Análise com SQL Avançado
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [2]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

In [3]:
SELECT * FROM Crop_Group LIMIT 10;

In [4]:
SELECT * FROM FCID_Description LIMIT 10;

In [5]:
SELECT * FROM Recipes LIMIT 10;

In [6]:
SELECT * FROM Intake LIMIT 10;

# Métricas

Considere que a tabela Intake registra alimentos consumidos por 1.489 pessoas. Considere as seguintes métricas para um alimento:

| Métrica | Descrição |
| --- | --- |
| Popularidade | número de pessoas (dentre as 1.489) que consumiram o alimento |
| Intake_Sum | total consumido do alimento pelas 1.489 pessoas em gramas |
| Intake_AVG | média de consumo do alimento em gramas |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |
| Recipes | número de receitas (dentre as 7.154 receitas) que têm o produto como ingrediente |

## 1)  Construa uma View que apresente essas métricas por produto

* Veja exemplo em: `/data/food-intake/computed/commodity-profile.csv`
* Importante: esta tabela foi feita com um número maior de registros, portanto os valores não serão iguais aos seus

In [ ]:
DROP VIEW IF EXISTS MetricasFinais;
DROP VIEW IF EXISTS MetricasAlimentos;
DROP VIEW IF EXISTS ReceitasIngredientes;

-- Metricas relacionadas ao consumo e às quantidades
CREATE VIEW MetricasAlimentos AS
    SELECT FCID_CODE, COUNT(*) POPULARITY, SUM(Consumption) INTAKE_SUM,
           SUM(Consumption) / SUM(TimesConsumed) INTAKE_AVG,
           SUM(ConsumptionBW) / SUM(TimesConsumed) INTAKE_BW_AVG
    FROM
        (SELECT FCID_CODE, SeqN, SUM(Intake) Consumption, SUM(Intake_BW) ConsumptionBW, COUNT(*) TimesConsumed
         FROM Intake
         GROUP BY FCID_CODE, SeqN) Alimento
    GROUP BY FCID_CODE;

-- Quantidade de vezes que cada ingrediente aparece em cada receita
CREATE VIEW ReceitasIngredientes AS
    SELECT FCID_Code, COUNT(*) RECIPES
    FROM
        (SELECT FCID_Code
        FROM Recipes
        GROUP BY FCID_Code, Food_Code, Mod_Code) Receitas
    GROUP BY FCID_Code;

-- VIEW final, incluindo dados sobre alimentos e métricas (resposta da questão)
CREATE VIEW MetricasFinais AS
    SELECT F.FCID_CODE, F.FCID_DESC, F.CGN, F.CG_SUBGROUP,
           MA.POPULARITY, MA.INTAKE_SUM, MA.INTAKE_AVG, MA.INTAKE_BW_AVG,
           RI.RECIPES
    FROM FCID_Description F, MetricasAlimentos MA, ReceitasIngredientes RI
    WHERE F.FCID_Code = MA.FCID_Code AND F.FCID_Code = RI.FCID_Code;

In [ ]:
-- Metricas relacionadas ao consumo e às quantidades
SELECT * FROM MetricasAlimentos ORDER BY POPULARITY DESC LIMIT 10;

In [ ]:
-- Quantidade de vezes que cada ingrediente aparece em cada receita
SELECT * FROM ReceitasIngredientes ORDER BY RECIPES DESC LIMIT 10;

In [ ]:
-- VIEW final, incluindo dados sobre alimentos e métricas (resposta da questão)
SELECT * FROM MetricasFinais ORDER BY POPULARITY DESC, FCID_CODE LIMIT 10;

## 2) Como você analisaria a correlação entre as métricas?

* Por exemplo, produtos mais populares são mais consumidos (em número de pessoas ou em quantidade)?
* Proponha uma ou mais queries para fazer esta análise

In [ ]:
-- Uma análise que pensamos entre as métricas é "Os alimentos mais populares são os que mais aparecem em receitas?"
-- Para respondermos a essa pergunta, pensamos em comparar os valores obtidos na ordenação por popularidade e na ordenação por receitas

SELECT FCID_DESC, POPULARITY, RECIPES FROM MetricasFinais ORDER BY Popularity DESC LIMIT 10;
SELECT FCID_DESC, POPULARITY, RECIPES FROM MetricasFinais ORDER BY Recipes DESC LIMIT 10;

-- Analisando os resultados parciais dessas queries, observamos que 7 produtos mais populares também 
-- são os que mais aparecem em receitas, o que indica que essas duas métricas podem estar correlacionadas.

## 3) Podemos criar grupos de consumidores conforme um perfil?
* por exemplo, consumidores podem ser agrupados por alimentos que comem predominantemente?
* como você associaria grupos a classes?

In [ ]:
-- Podemos agrupar os consumidores com base na variedade de alimentos consumidos. 
-- Para cada consumidor diferente, podemos determinar quantos alimentos diferentes foram consumidos por ele. 
-- Com isso, poderíamos associar esses grupos a classes, por exemplo:
--   Classe A: até 30 alimentos diferentes consumidos
--   Classe B: de 30 até 80 alimentos diferentes consumidos
--   Classe C: 80 ou mais alimentos diferentes consumidos
-- Dados sobre as classes serão mostrados na questão seguinte

## 4) Que métricas podem ser analisadas para a comparação de perfis?
* escreva uma query SQL que calcule pelo menos uma métrica comparativa

In [ ]:
-- VIEW que representa o perfil de consumo de cada pessoa
DROP VIEW IF EXISTS PerfilConsumo;

CREATE VIEW PerfilConsumo AS
    SELECT SEQN, COUNT(DISTINCT FCID_CODE) VARIEDADE
        FROM Intake
        GROUP BY SEQN;

In [ ]:
-- O total de consumidores diferentes é:
SELECT COUNT(*) FROM PerfilConsumo;

In [ ]:
-- Os tamanhos das classes identificadas são:
SELECT COUNT(*) FROM PerfilConsumo WHERE VARIEDADE < 40;
SELECT COUNT(*) FROM PerfilConsumo WHERE VARIEDADE >= 40 AND VARIEDADE < 80;
SELECT COUNT(*) FROM PerfilConsumo WHERE VARIEDADE >= 80;

-- Assim, podemos relacionar esses números, extraindo os dados de que 27% dos consumidores são da Classe A,
--   69% dos consumidores são da Classe B e 4% da classe C